In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
with open('authorized.txt', 'w') as a:
    a.write('True')

In [4]:
import streamlit as st
from multipage_template import save, MultiPage, start_app, clear_cache
import pandas as pd
from PIL import Image
import os
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
import random
from sklearn.decomposition import PCA
import math
from spotifyAPI import Spotify

In [5]:
userOneFeatures = pd.read_csv("../userOneFeatures.csv")
userTwoFeatures = pd.read_csv("../userTwoFeatures.csv")
userThreeFeatures = pd.read_csv("../userThreeFeatures.csv")

userOneFeatures.drop('Unnamed: 0', axis = 1, inplace = True)
userTwoFeatures.drop('Unnamed: 0', axis = 1, inplace = True)
userThreeFeatures.drop('Unnamed: 0', axis = 1, inplace = True)

In [6]:
s = Spotify()

Client ID:


 ································


Client Secret:


 ································


In [9]:
import json

In [10]:
def get_X(playlist = 'Top 50 - USA', creator = 'Spotify', playlist_id = ""):
    '''
    Takes in a playlis,m ;l909t name and the creator of the playlist and
    returns the list of songs in that playlist. Also returns the features.

    Params:
    playlist (string) - the name of the playlist (default Top 50)
    creator (string) - the creator of the playlist (default Spotify)

    Returns:
    songs (list) - a list of song names
    X (list of dictionaries) - our features
    '''
    if os.path.exists('X_songs.txt'):
        #if our song exists
        songs = open('X_songs.txt').read().split("~")
        if len(songs) <= 1:
            songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('X_songs.txt', 'w', encoding = 'utf-8') as s_file:
                for song in songs:
                    s_file.write(song[0] + "~")
                songs = open('X_songs.txt').read().split("~")
    else:
        songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('X_songs.txt', 'w', encoding = 'utf-8') as s_file:
            for song in songs:
                s_file.write(song[0] + "~")
            songs = open('X_songs.txt').read().split("~")

    if os.path.exists('X_features.json'):
        f = open('X_features.json')
        features = json.load(f)
        if len(features) <= 1:
            features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('X_features.json', 'w', encoding='utf-8') as f:
                json.dump(features, f, ensure_ascii=False, indent=4)
    else:
        features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('X_features.json', 'w', encoding='utf-8') as f:
            json.dump(features, f, ensure_ascii=False, indent=4)

    return songs[:len(songs)-1], features

def get_y(playlist, creator, playlist_id):
    '''
    Takes in a playlist name and the creator of the playlist and
    returns the list of songs in that playlist. Also returns the features.

    Params:
    playlist (string) - the name of the playlist
    creator (string) - the creator of the playlist

    Returns:
    songs (list) - a list of song names
    features (list of dictionaries) - our y features
    '''
    if os.path.exists('y_songs.txt'):
        #if our song exists
        songs = open('y_songs.txt').read().strip().split("~")
        if len(songs) <= 0:
            songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('y_songs.txt', 'w', encoding = 'utf-8') as s_file:
                for song in songs:
                    s_file.write(song[0] + "~")
                songs = open('y_songs.txt').read().strip().split("~")
    else:
        songs = s.get_playlist_items(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('y_songs.txt', 'w', encoding = 'utf-8') as s_file:
            for song in songs:
                s_file.write(song[0] + "~")
            songs = open('y_songs.txt').read().strip().split("~")

    if os.path.exists('y_features.json'):
        f = open('y_features.json')
        features = json.load(f)
        if len(features) <= 0:
            features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
            with open('y_features.json', 'w', encoding='utf-8') as f:
                json.dump(features, f, ensure_ascii=False, indent=4)
    else:
        features = s.get_playlist_features(query = playlist, desired_artist = creator, playlist_id = playlist_id)
        with open('y_features.json', 'w', encoding='utf-8') as f:
            json.dump(features, f, ensure_ascii=False, indent=4)

    return songs[:len(songs)-1], features

In [15]:
y_songs, y = get_y('Top 50', 'Spotify', '')

In [16]:
X_songs, X = get_X('Christmas Hits', 'Spotify')

In [17]:
from recommender import songRecommender

In [18]:
model = songRecommender(X, y, X_songs, y_songs)
#you can also flip it to get the top 50 songs and their corresponding matches

In [19]:
X = model.getX()
y = model.getY()

In [20]:
len([i[0] for i in X])

50

In [21]:
len([i[0] for i in y])

100

In [22]:
pd.Data(y_songs)

['All I Want for Christmas Is You',
 'Last Christmas',
 'Santa Tell Me',
 "Rockin' Around The Christmas Tree - Single Version",
 "It's Beginning to Look a Lot like Christmas",
 'Jingle Bell Rock',
 'Underneath the Tree',
 'Let It Snow, Let It Snow, Let It Snow',
 "It's the Most Wonderful Time of the Year",
 'Snowman',
 'Mistletoe',
 'Sleigh Ride',
 'Holly Jolly Christmas',
 'Feliz Navidad',
 'Wonderful Christmastime (Edited Version) [Remastered]',
 'Christmas (Baby Please Come Home)',
 'Driving Home for Christmas',
 'Let It Snow! Let It Snow! Let It Snow! (with The B. Swanson Quartet)',
 'Jingle Bell Rock',
 "Do They Know It's Christmas? - 1984 Version",
 'Merry Christmas Everyone - Remastered',
 'Have Yourself A Merry Little Christmas',
 "It's Beginning to Look a Lot Like Christmas (with Mitchell Ayres & His Orchestra)",
 'Last Christmas',
 "Baby, It's Cold Outside - Remastered 2004",
 'Happy Xmas (War Is Over) - Remastered 2010',
 'Santa Baby',
 "Thank God It's Christmas",
 'A Holly 